In [11]:
print(cursor.execute(f"SELECT * from personnages where patronus like '%Nar%'").fetchall())

[('Alastor Maugrey', 'Homme', "Un œil petit et sombre et un autre grand, rond, d'un bleu électrique", 'Gris', 1961, 1997, 'Canard', 'Sorcier'), ('Seamus Finnigan', 'Homme', 'Violets', 'Blonds', 1979, 0, 'Renard', 'Sang-mêlé'), ('Hugo Granger-Weasley', 'Homme', 'Gris', 'Bruns', 0, 0, 'Canard', 'Sang-mêlé'), ('Gilderoy Lockhart', 'Homme', 'Bleu clair', 'Blonds ondulés', 1964, 0, 'Canard', 'Sang-mêlé'), ('Narcissa Malefoy', 'Femme', 'Bleus', 'Blonds', 1955, 0, 'Canard', 'Sang-pur'), ('Charles Weasley', 'Homme', 'Marron', 'Roux', 1972, 0, 'Canard', 'Sang-pur'), ('Vernon Dursley', 'Homme', 'Marrons', 'Noirs', 0, 0, 'Canard', 'Moldu'), ('Millicent Bagnold', 'Femme', 'Gris', 'Noirs', 0, 0, 'Canard', 'Sorcière'), ('Wilkie Tycross', 'Homme', 'Noirs', 'Roux', 0, 0, 'Canard', 'Sorcier'), ('Charity Burbage', 'Femme', 'Bruns', 'Bruns', 0, 1997, 'Canard', 'Sorcière'), ('Joey Jenkins', 'Homme', 'Bleus', 'Roux', 0, 0, 'Canard', 'Sorcier'), ('Frank Bryce', 'Homme', 'Verts', 'Bruns', 1917, 1994, 'Canard

In [ ]:
with open('personnages.txt','x') as f:
    for page in pages:
        if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
            f.write(xmlToSql.getAttribute("Nom",page) +" "+ xmlToSql.getAttribute("Yeux",page)+"\n")

In [1]:
from bs4 import BeautifulSoup
import re
from xmlToSql import getAttribute
import sqlite3

Reading and parsing the dump with BeautifulSoup

In [2]:
with open('frharrypotter_pages_current.xml','r') as f:
    file = f.read()

soup = BeautifulSoup(file, 'xml')
pages = soup.find_all('page')

## Creating the databases

Start by creating an empty database.

In [3]:
connection = sqlite3.connect("harrypotter_fr.db")
cursor = connection.cursor()

### First the character table

Create the table for characters named "personnages".

Fill in the records thanks to the dump of the french Harry Potter Fandom and parsing functions in the xmlToSql.py file.

In [4]:
cursor.execute("CREATE TABLE personnages (nom TEXT NOT NULL PRIMARY KEY, genre TEXT, yeux TEXT, cheveux TEXT, naissance INTEGER, mort INTEGER, patronus TEXT, sang TEXT)")
for page in pages:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text):
        nom = getAttribute("Nom",page)
        yeux = getAttribute("Yeux",page)
        cheveux = getAttribute("Cheveux",page)
        genre = getAttribute("Genre",page)
        patronus = getAttribute("Patronus",page)
        naissance = getAttribute("Naissance",page)
        mort = getAttribute("Mort",page)
        sang = getAttribute("Sang",page)
        # print(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        try:
            if nom =='?':
                continue
            cursor.execute(f"Insert INTO personnages VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        except:
            print(f"Already inserted {nom}")

Already inserted Lily Potter
Already inserted Fred Weasley
Already inserted Molly Weasley
Already inserted Avery
Already inserted Mulciber
Already inserted Élève garçon de Beauxbâtons non identifié
Already inserted Élève garçon de Beauxbâtons non identifié
Already inserted Élèves filles de Beauxbâtons non identifiées
Already inserted Stebbins
Already inserted Emily
Already inserted Derek
Already inserted M. Carneirus
Already inserted Stebbins
Already inserted Julia Dehoff
Already inserted M. Carneiro
Already inserted Emma Vane
Already inserted Luca Caruso
Already inserted Luca Caruso
Already inserted Albert
Already inserted Andrew Snowy Owl
Already inserted Winky Crockett
Already inserted Jo King
Already inserted Emma Vanity
Already inserted L. Wakefield
Already inserted William de Cognac
Already inserted Eduardo Lima
Already inserted Eduardus Limus
Already inserted Miraforum
Already inserted Loretta Fieldwake
Already inserted M. Mina
Already inserted R. Amorim


### Then table for creatures

In [5]:
cursor.execute("CREATE TABLE créatures (nom_créature TEXT NOT NULL PRIMARY KEY, statut_créature TEXT, yeux_créature TEXT, plumes_poils text, apparence TEXT, caractère TEXT, localisation TEXT)")

In [6]:
for page in pages:
    if "[[Catégorie:Créature" in page.text and ("Catégorie:" not in page.title.text and "Portail:" not in page.title.text):
        nom = getAttribute("Nom",page)
        yeux = getAttribute("Yeux",page)
        poils = getAttribute("Plumes/Poils",page)
        statut = getAttribute("Statut",page)
        apparence = getAttribute("Apparence",page)
        caractère = getAttribute("Caractère",page)
        localisation = getAttribute("Localisation",page)
        #print(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{apparence}','{caractère}','{localisation}')")
        try:
            if nom =='?':
                continue
            cursor.execute(f"Insert INTO créatures VALUES('{nom}','{statut}','{yeux}','{poils}','{apparence}','{caractère}','{localisation}')")
        except:
            print(f"Already inserted {nom}")

Already inserted Diablotin
Already inserted Cachalot
Already inserted Hippocampe


## Finally Junction Table between family members

In [7]:
cursor.execute("CREATE TABLE famille (premier_nom TEXT NOT NULL, second_nom TEXT NOT NULL, relation TEXT, FOREIGN KEY (premier_nom) REFERENCES personnages(nom), FOREIGN KEY (second_nom) REFERENCES personnages(nom))")

In [8]:
def insertRelative(first,second,relation):
    first = first.replace("'","''")
    second = second.replace("'","''")
    relation = relation.replace("'","''").capitalize().strip()
    try:
        if first =='?' or second == '?':
            return
        cursor.execute(f"Insert INTO famille VALUES('{first}','{second}','{relation}')")
    except:
        print(f"Inserting {first} {second} {relation} didn't work")
    return

pattern = re.compile(r'Famille\s*=\s*(.*?)\s\|', re.S)
for page in pages:
    if "[[Catégorie:Personnage]]" in page.text and ("Catégorie:" not in page.title.text and "Utilisateur:" not in page.title.text):
        nom = getAttribute("Nom",page)
        if re.search('Famille\s*=\s*\n\|',page.text): #Related to no-one
            continue
        singleRelative = re.search('Famille\s*=\s*\[\[(.*?)\]\]',page.text)
        if singleRelative:#Related to one person
            onlyRelative = singleRelative.group(1)
            if "|" in onlyRelative:
                parts = onlyRelative.split("|")
                insertRelative(parts[0],nom,parts[1])
            else:
                relation = re.search('Famille=.*?{{Comment\|(.*?)}}',page.text)
                if relation:
                    insertRelative(onlyRelative,nom,relation.group(1))
                else:
                    insertRelative(onlyRelative,nom,"?")
            continue
        multipleRelatives = pattern.search(page.text)
        if multipleRelatives:
            famille_text = multipleRelatives.group(1)
            famille = re.findall(r'\*\[\[([^\]]+)\]\]',famille_text)
            for member in famille:
                if "|" in member:
                    parts = member.split("|")
                    insertRelative(parts[0],nom,parts[1])
                else:
                    relation = re.search(f'\*\[\[{member}\]\].*?{{{{Comment\|(.*?)}}}}',page.text)
                    if relation:
                        insertRelative(member,nom, relation.group(1))
            
        

Finally commit the changes to the database so that all changes are saved and close the connection

In [12]:
connection.commit()
cursor.close()
connection.close()

# Jetzt auf Deutsch

In [1]:
from bs4 import BeautifulSoup
import re
from xmlToSql_de import getAttribute, randomly_attribute
import sqlite3
import random

In [2]:
with open('deharrypotter_pages_current.xml','r') as f:
    file = f.read()

soup = BeautifulSoup(file, 'xml')
pages = soup.find_all('page')

In [6]:
connection = sqlite3.connect("harrypotter_de.db")
cursor = connection.cursor()

In [7]:
cursor.execute("CREATE TABLE charaktere (Name TEXT NOT NULL PRIMARY KEY, Geschlecht TEXT, Augen TEXT, Haare TEXT, Geburt INTEGER, Tod INTEGER, Patronus TEXT, Blutstatus TEXT)")
kategorien =["[[Kategorie:Weibliche Charaktere]]", "Kategorie:Hexen / Zauberer", "Kategorie:Männliche Charaktere"]
for page in pages:
    if any(kategorie in page.text for kategorie in kategorien) and (":" not in page.title.text):
        nom = getAttribute("Name",page)
        if nom == "?":
            nom = page.title.text
        if not "Charakterinfobox" in page.text and not "Infobox" in page.text :
            nom = page.title.text
            yeux = randomly_attribute("Augen")
            cheveux = randomly_attribute("Haare")
            genre = randomly_attribute("Geschlecht")
            naissance = 0
            mort = 0
            sang = "?"
            patronus = randomly_attribute("Patronus")
        else:
            yeux = getAttribute("Augen",page)
            cheveux = getAttribute("Haare",page)
            genre = getAttribute("Geschlecht",page)
            patronus = getAttribute("Patronus",page)
            naissance = getAttribute("Geburt",page)
            mort = getAttribute("Tod",page)
            sang = getAttribute("Blut",page)
        if (nom == "?"):
            print(page.title.text)
        #cursor.execute(f"Insert INTO charaktere VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        try:
            # print(f"Insert INTO personen VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
            cursor.execute(f"Insert INTO charaktere VALUES('{nom}','{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}')")
        except:
            print(f"Already inserted {nom},'{genre}','{yeux}','{cheveux}','{naissance}','{mort}','{patronus}','{sang}'")

Already inserted Bartemius Crouch,'Männlich','Silbergrau','Grau','1964','1995','Phönix','Reinblut'
Already inserted Sirius Black,'Männlich','Braun','Blond','0','0','Pufferfisch','Reinblut'
Already inserted Fred Weasley,'Männlich','Braun','Rot','1978','1998','Elster','Reinblut'
Already inserted Cygnus Black,'Männlich','Braun','Schwarz','0','0','Otter','Reinblut'
Already inserted Sirius Black,'Männlich','Blau','Braun','0','0','Kniesel','Reinblut'
Already inserted Arcturus Black,'Männlich','Rot','Braun','0','0','Drachen','Reinblut'
Already inserted Cygnus Black,'Männlich','Schwarz','Braun','0','0','Kniesel','Reinblut'
Already inserted Arcturus Black,'Männlich','Grün','Grau','0','0','Pufferfisch','Reinblut'
Already inserted Riese in der Schlacht von Hogwarts,'Männlich','Blau','Schwarz, glatzenbildung','0','1998','Einhorn','?'
Already inserted Jo King,'Weiblich','Braun','Schwarz','0','0','Löwe','?'
Already inserted Irving Cram,'Männlich','Grün','Schwarz','1985','0','Bowtruckle','?'
Already 

In [8]:
cursor.execute(f"UPDATE charaktere SET Name = 'Humphrey Belcher' WHERE name like '%Humphrey Belcher%' ")
cursor.execute(f"UPDATE charaktere SET Blutstatus = 'Maledictus' WHERE name = 'Nagini' ")
cursor.execute(f"UPDATE charaktere SET Patronus = 'Löwe' WHERE name = 'Der Blutige Baron' ")
cursor.execute(f"UPDATE charaktere SET Patronus = 'Hirsch' WHERE name = 'Severus Snape' ")
cursor.execute(f"UPDATE charaktere SET Patronus = 'Hirsch' WHERE name = 'Harry James Potter' ")
cursor.execute(f"UPDATE charaktere SET Patronus = 'Damhirschkuh' WHERE name = 'Lily J. Potter (geb. Evans)' ")
cursor.execute(f"UPDATE charaktere SET Name = 'Bellatrix Lestrange' WHERE name = 'Bellatrix Lestrange (geb. Black)' ")
cursor.execute(f"UPDATE charaktere SET Geburt = 1850, Tod = 1931 WHERE name = 'Elladora Black' ")
cursor.execute(f"UPDATE charaktere SET Geburt = 1904, Tod = 1965 WHERE name = 'Lycoris Black' ")
cursor.execute(f"UPDATE charaktere SET Geburt = 1925, Tod = 1985, Patronus='Hase' WHERE name = 'Walburga Black' ")
cursor.execute(f"UPDATE charaktere SET Geschlecht = 'Weiblich', Geburt = 1961 WHERE name = 'Andromeda Tonks' ")
cursor.execute(f"UPDATE charaktere SET Geschlecht = 'Weiblich', Geburt = 1908, Haare='Blond' WHERE name = 'Irma Crabbe' ")
print(cursor.execute(f"SELECT * from charaktere WHERE name = 'Humphrey Belcher'").fetchall())

[('Humphrey Belcher', 'Männlich', 'Blau', 'Schwarz', 0, 0, 'Diricawl', '?')]


## Kreaturen

In [9]:
cursor.execute("CREATE TABLE kreaturen (name_kreatur TEXT NOT NULL PRIMARY KEY, status_kreatur TEXT, augen_kreatur TEXT, haut_kreatur text, federn_kreatur, eigenschaften_kreatur TEXT, verbreitung_kreatur TEXT)")

In [10]:
kategorien = ["Kategorie:Tierwesen","Kategorie:Vögel","Kategorie:Drachen",
              "Kategorie:Kreatur","Kategorie:XXX Kreaturen","Kategorie:Flugfähige Kreaturen",
              "Kategorie:Verstorbene Kreaturen","Kategorie:Sprechende Kreaturen","Kategorie:Magische Kreaturen"
             "Kategorie:XX Kreaturen","Kategorie:X Kreaturen","Kategorie:XXXX Kreaturen","Kategorie:XXXXX Kreaturen"]
for page in pages:
    if any(kategorie in page.text for kategorie in kategorien) and ("Kategorie:" not in page.title.text and "Portal:" not in page.title.text and "Vorlage:" not in page.title.text):
        if not "Charakterinfobox" in page.text and not "Infobox" in page.text:
            continue
        else:
            nom = getAttribute("Name",page)
            yeux = getAttribute("Augen",page)
            plumes = getAttribute("Federn",page)
            peau = getAttribute("Haut",page)
            statut = getAttribute("Status",page)
            spezies = getAttribute("Spezies",page)
            statut = statut if (not statut=="?") else spezies
            eigenschaft = getAttribute("Eigenschaft",page)
            localisation = getAttribute("Verbreitung",page)
        # print(f"Insert INTO kreaturen VALUES('{nom}','{statut}','{yeux}','{peau}','{plumes}','{eigenschaft}','{localisation}')")
        # cursor.execute(f"Insert INTO kreaturen VALUES('{nom}','{statut}','{yeux}','{peau}','{plumes}','{eigenschaft}','{localisation}')")
        try:
            cursor.execute(f"Insert INTO kreaturen VALUES('{nom}','{statut}','{yeux}','{peau}','{plumes}','{eigenschaft}','{localisation}')")
        except:
            print(f"Already inserted '{nom}','{statut}','{yeux}','{peau}','{plumes}','{eigenschaft}','{localisation}'")

In [11]:
cursor.execute(f"UPDATE kreaturen SET status_kreatur = 'Maledictus'  WHERE Name_Kreatur='Nagini'")

## Familien

In [12]:
cursor.execute("CREATE TABLE verwandte (erster_name TEXT NOT NULL, zweiter_name TEXT NOT NULL, verwandtschaft TEXT, FOREIGN KEY (erster_name) REFERENCES charaktere(name), FOREIGN KEY (zweiter_name) REFERENCES charaktere(name))")

In [13]:
def insertRelative(first,second,relation):
    #print(f"Insert INTO verwandte VALUES('{first}','{second}','{relation}')")
    first = first.replace("'","''").replace('[', '').replace(']', '').replace('ß','ss')
    second = second.replace("'","''").replace('[', '').replace(']', '').replace('ß','ss')
    relation = relation.replace("'","''").replace('[', '').replace(']', '').replace('ß','ss')
    try:
        if first =='?' or second == '?':
            return
        #print(f"Insert INTO verwandte VALUES('{first}','{second}','{relation}')")
        cursor.execute(f"Insert INTO verwandte VALUES('{first}','{second}','{relation}')")
    except:
        print(f"Inserting {first} {second} {relation} didn't work")
    return

pattern = re.compile(r'Familie\s*=\s*(.*?)\s\|', re.S)
kategorien =["[[Kategorie:Weibliche Charaktere]]", "Kategorie:Hexen / Zauberer", "Kategorie:Männliche Charaktere"]
for page in pages:
    if any(kategorie in page.text for kategorie in kategorien) and ("Kategorie:" not in page.title.text):
        nom = getAttribute("Name",page)
        if nom == "?":
            nom = page.title.text
        if re.search('Familie\s*=\s*\n\|',page.text): #Related to no-one
            continue
        singleRelative = re.search('Familie\s*=\s*\[\[(.*?)\]\]',page.text)
        if singleRelative:#Related to one person
            onlyRelative = singleRelative.group(1)
            relation = re.search('Familie=.*?\((.*?)\)',page.text)
            if relation:
                insertRelative(onlyRelative,nom,relation.group(1))
            else:
                insertRelative(onlyRelative,nom,"?")
            continue
        multipleRelatives = pattern.search(page.text)
        if multipleRelatives:
            famille_text = multipleRelatives.group(1)
            famille = re.findall(r'\*\[\[([^\]]+)\]\]',famille_text)
            for member in famille:
                search_member = member.replace('|','\|')
                relation = re.search(f'\*\[\[{search_member}\]\].*?\((.*?)\)',page.text)
                if relation:
                    insertRelative(member,nom, relation.group(1))
                else:
                    insertRelative(member,nom, "?")    
                    
insertRelative('Narzissa Malfoy','Lucius Malfoy','Ehefrau')
insertRelative('Lucius Malfoy','Narzissa Malfoy','Ehemann')
insertRelative('Bellatrix Lestrange (geb. Black)','Sirius Black','Cousine mütterlicherseits')
cursor.execute(f"UPDATE verwandte SET erster_Name = 'Bellatrix Lestrange' WHERE erster_name = 'Bellatrix Lestrange (geb. Black)' ")
cursor.execute(f"UPDATE verwandte SET zweiter_Name = 'Bellatrix Lestrange' WHERE zweiter_name = 'Bellatrix Lestrange (geb. Black)' ")

In [14]:
print(cursor.execute(f"SELECT * FROM verwandte LIMIT 15").fetchall())

[('Ignotus Peverell', 'Albus Severus Potter', 'väterlicher Vorfahre'), ('Linfred of Stinchcombe', 'Albus Severus Potter', 'väterlicher Vorfahre'), ('James Potter I|James Potter', 'Albus Severus Potter', 'Grossvater väterlicherseits'), ('Lily J. Potter|Lily Potter', 'Albus Severus Potter', 'Grossmutter väterlicherseits'), ('Arthur Weasley', 'Albus Severus Potter', 'Grossvater mütterlicherseits'), ('Molly Weasley', 'Albus Severus Potter', 'Grossmutter mütterlicherseits'), ('Harry Potter', 'Albus Severus Potter', 'Vater'), ('Ginevra Weasley|Ginny Potter', 'Albus Severus Potter', 'Mutter'), ('James Potter II', 'Albus Severus Potter', 'Bruder'), ('Lily Potter II', 'Albus Severus Potter', 'Schwester'), ('Ronald Weasley|Ron Weasley', 'Albus Severus Potter', 'Onkel mütterlicherseits'), ('Hermine Granger|Hermine Weasley', 'Albus Severus Potter', 'angeheiratete Tante'), ('Rose Granger-Weasley', 'Albus Severus Potter', 'mütterliche erste Cousine'), ('Hugo Granger-Weasley', 'Albus Severus Potter',

In [15]:
connection.commit()
cursor.close()
connection.close()

In [6]:
target = pages[0].text
print(target)
ttt = "in Erinnerung an den großen Magier"
ttt=target.replace('ß','ss').replace('den','zbra')
print(ttt)


Datei:Lexikon.png
6
1

45748
44745
2014-07-03T22:03:42Z

Harry granger
1590788


45748
wikitext
text/x-wiki
{{Information
|Beachten=
|Beschreibung=Logo für de.harrypotter.wikia.com, siehe auch [[:Datei:Wiki wide.png|Wiki wide.png]]
|Quelle=[[w:c:harrypotter:File:Wiki.png|harrypotter.wikia.com]]
|Autor=erstellt von [[w:c:harrypotter:User:Vostok|Vostok]] (wahrscheinlich)
|Dateispezis=
|Lizenz=[http://www.gnu.org/copyleft/fdl.html GFDL]
|Andere Versionen=
|Kategorien=
}}
:''Diese Datei ist lizenziert unter GFDL. Erlaubnis wurde erteilt für das Kopieren, Veröffentlichen und/oder Bearbeiten dieser Datei gemäß den Bedingungen der GNU Free Documentation License, Version 1.2 oder jede spätere Version, veröffentlicht durch die Free Software Foundation; ohne unveränderliche Abschnitte, ohne Vorderdeckel-Texte und Rückendeckel-Texte.''

:''This file is licensed under the GFDL. Permission is granted to copy, distribute and/or modify this image under the terms of the GNU Free Documentation License